In [1]:
# RUN THIS CELL TO STORE ON MINERS USING A VALIDATOR HOTKEY
import sys
import base64
import random
import numpy as np
import bittensor as bt
from redis import asyncio as aioredis
from Crypto.Random import get_random_bytes
from storage import protocol
from storage.shared.utils import get_redis_password
from storage.validator.database import *
from storage.shared.ecc import (
    hash_data,
    setup_CRS,
    ecc_point_to_hex,
)

data = b"Some bytes data to store on the network!"
subtensor = bt.subtensor("test")
metagraph = subtensor.metagraph(netuid=22, lite=False)

# Grab top 10% of miners by incentive
def get_top_miner_uids(n=0.1):
    top_i = np.quantile(metagraph.I, 1 - n)
    uids = metagraph.uids[metagraph.I > top_i]
    return [uid.item() for uid in uids]

axons = [metagraph.axons[uid] for uid in get_top_miner_uids()]

# Setup CRS (common reference string) for this round of validation
curve = "P-256"
g, h = setup_CRS(curve=curve)

# Hash the data
data_hash = hash_data(data)

# Convert to base64 for compactness
b64_encrypted_data = base64.b64encode(data).decode("utf-8")

# Create Store synapse
synapse = protocol.Store(
    encrypted_data=b64_encrypted_data,
    curve=curve,
    g=ecc_point_to_hex(g),
    h=ecc_point_to_hex(h),
    seed=get_random_bytes(32).hex(),  # 256-bit seed
    ttl=12345, # how many seconds before miners can safely delete
)

# Save metadata for later (this is taken care of in the redis db internally)
data = {
    "data_hash": data_hash,
    "curve": curve,
    "prev_seed": synapse.seed,
    "ttl": synapse.ttl,
    "size": sys.getsizeof(data),
}

# NOTE: Must be a registered validator hotkey to work.
name = "default"
hotkey = "default"
wallet = bt.wallet(name, hotkey)
dendrite = bt.dendrite(wallet)

responses = await dendrite(
    axons,
    synapse,
    deserialize=False
)

responses

2024-04-17 14:49:56.670 |       INFO       | Connected to test network and wss://test.finney.opentensor.ai:443/.
2024-04-17 14:49:58.907 |      TRACE       | dendrite | --> | 5025 B | Store | 5GjBkhRhq5V7yiqdPDFe3k8rj3CsrfW7r6u7S6FKeYJ8627i | 149.137.225.62:11114 | 0 | Success
2024-04-17 14:49:58.935 |      TRACE       | dendrite | --> | 5025 B | Store | 5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz | 149.137.225.62:11113 | 0 | Success
2024-04-17 14:49:58.956 |      TRACE       | dendrite | --> | 5025 B | Store | 5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2 | 149.137.225.62:11118 | 0 | Success
2024-04-17 14:49:58.970 |      TRACE       | dendrite | --> | 5025 B | Store | 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU | 149.137.225.62:11112 | 0 | Success
2024-04-17 14:49:58.983 |      TRACE       | dendrite | --> | 5025 B | Store | 5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK | 149.137.225.62:11124 | 0 | Success
2024-04-17 14:49:58.995 |      TRACE       | dendrite | --> | 

[Store(required_hash_fields=['curve', 'g', 'h', 'seed', 'randomness', 'commitment', 'signature', 'commitment_hash'], encrypted_data='', curve='P-256', g='3131313235353337363635313935363636323237303833363835343635373633393633393137303137323537353732303138383734333939323036383834393138383233323630343632383433382c3231383734363739343635303538303231323039303638393634393438373938333035373932323730363836333337383834363435373236363635383535343438393630363233383037373638', h='34313935323735353630363333323938313631363535353336313133313439303130353733303131303233383631343831373835383237393035323230333932353233323736333639303332382c3933353737363134343439323831383536363237303133363830353834303133353238313436373230393334373736343736373236353133363930373732363637343831363136383634383439', seed='fe11a58b635b4adfb3651a0eedac80eb4f61b4dc69dea478f9f40f1c1c524c44', randomness=46696406650170129862424594513419296712123841749714442359667968656796036783829, commitment='3138383630303935363834373835353236303630

In [15]:
# CHALLENGE THOSE SAME MINERS WITH THE DATA HASH WE JUST USED
import random
from storage.validator.verify import verify_challenge_with_seed

min_chunk_size = 24 # bytes
chunk_factor = 4
chunk_size = 2 # bytes
bt.logging.trace(f"Challenge lookup key: {data_hash}")

num_chunks = (
    data["size"] // chunk_size if data["size"] > chunk_size else data["size"]
)
print("num chunks:", num_chunks)
# Setup new Common-Reference-String for this challenge
g, h = setup_CRS()

synapse = protocol.Challenge(
    challenge_hash=data_hash,
    chunk_size=chunk_size,
    g=ecc_point_to_hex(g),
    h=ecc_point_to_hex(h),
    curve=data["curve"],
    challenge_index=random.choice(range(num_chunks)),
    seed=get_random_bytes(32).hex(),
)

# Reuse axons from previous cell that we stored data with
responses = await dendrite(
    axons,
    synapse,
    deserialize=True,
    timeout=30,
)

for response in responses:
    # Verify the response
    verified = verify_challenge_with_seed(response, synapse.seed)

    if verified:
        print(f"Yay! Verified response from miner {response.axon.hotkey}")


num chunks: 36
Yay! Verified response from miner 5GjBkhRhq5V7yiqdPDFe3k8rj3CsrfW7r6u7S6FKeYJ8627i
Yay! Verified response from miner 5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz
Yay! Verified response from miner 5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2
Yay! Verified response from miner 5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU
Yay! Verified response from miner 5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK
Yay! Verified response from miner 5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX
Yay! Verified response from miner 5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1
Yay! Verified response from miner 5EZCtyfQrDUQqRKYDdq1eaKSB9Ugkr3GxMb8XadewMRMxdev
Yay! Verified response from miner 5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon
Yay! Verified response from miner 5EXXF13sRaEzqqWqKm23GBWLZcnq4JTFAabZBui1iUKppcjp
Yay! Verified response from miner 5FpeAq7bEg1m7dEv1XF23SwHsEJEtHQQHpT5nt7qcBnSXmPt
Yay! Verified response from miner 5Dr1n6fCdUFQsZmJxhi2MH1hPeYwtcAAz54pKK

In [ ]:
# RUN THIS CELL TO STORE ON TESTNET
import time
import random
import bittensor as bt
from storage.api import store, retrieve
from storage.api import StoreUserAPI, RetrieveUserAPI, get_query_api_axons
bt.trace()

# setup wallet and subtensor connection
wallet_name = "default"
wallet_hotkey = "default"
wallet = bt.wallet(wallet_name, wallet_hotkey)
subtensor = bt.subtensor("test")

# Store some data and retrieve it
data = b"This is a test of the API high level abstraction"

print("Storing data on the Bittensor testnet.")
cid, hotkeys = await store(data, wallet, subtensor, netuid=22)

print("Stored {} with {} hotkeys".format(cid, hotkeys))

In [ ]:
# RUN THIS CELL TO RETRIEVE ON TESTNET
print("Now retrieving data with CID: ", cid)
rdata = await retrieve(cid, wallet, subtensor, netuid=22, hotkeys=hotkeys)
print(rdata)
assert data == rdata, "Data does not match!"

In [ ]:
# RUN THIS CELL TO STORE ON MAINNET (Assumes at least 1 API node is running with --open_access for whitelisting all keys.)

# setup wallet and subtensor connection
wallet_name = "default"
wallet_hotkey = "default"
wallet = bt.wallet(wallet_name, wallet_hotkey)
subtensor = bt.subtensor("finney")

# Store some data and retrieve it
data = b"This is a test of the API high level abstraction"

print("Storing data on the Bittensor testnet.")
cid, hotkeys = await store(data, wallet, subtensor, netuid=21)

print("Stored {} with {} hotkeys".format(cid, hotkeys))

In [ ]:
# RUN THIS CELL TO RETRIEVE ON MAINNET
print("Now retrieving data with CID: ", cid)
rdata = await retrieve(cid, wallet, subtensor, netuid=21, hotkeys=hotkeys)
print(rdata)
assert data == rdata, "Data does not match!"

In [ ]:
# STORE ON A SPECIFIC VALIDATOR
# setup wallet and subtensor connection
wallet_name = "default"
wallet_hotkey = "default"
wallet = bt.wallet(wallet_name, wallet_hotkey)
subtensor = bt.subtensor("finney")
validator_uid = 86

# Store some data and retrieve it
data = b"This is a test of the API high level abstraction"

print("Storing data on the Bittensor testnet.")
cid, hotkeys = await store(data, wallet, subtensor, netuid=21, uid=validator_uid)

print("Stored {} with {} hotkeys".format(cid, hotkeys))